# 7.1 HTTP协议基础与Web框架入门

## 📚 用途说明
本笔记本介绍HTTP协议基础知识和Web框架概念，为后续的异步Web开发奠定基础。结合已掌握的异步编程知识，理解现代Web框架的设计原理。

### 🎯 学习目标
- 掌握HTTP协议的核心概念和工作原理
- 理解Web框架的作用和设计模式
- 学习同步与异步Web框架的区别
- 为FastAPI等异步框架学习做好准备

### ⚡ 前置要求
- Python基础语法和面向对象编程
- 异步编程基础（协程、事件循环）
- 网络编程基础概念

### 🔄 LangChain关联
HTTP协议和Web框架是构建LangChain应用的基础：
- API服务：为LangChain提供HTTP接口
- Web应用：构建基于LangChain的对话系统
- 异步处理：提升LangChain应用的并发性能

---

In [ ]:
# 导入必要的标准库
import asyncio
import socket
import json
import time
from typing import Dict, Any, Optional, Tuple, List
from dataclasses import dataclass
from urllib.parse import urlparse, parse_qs
import email.message
from email.message import Message

print("🌐 HTTP协议基础与Web框架入门")
print("=" * 50)
print(f"📝 学习目标: 掌握HTTP协议和Web框架基础")
print(f"⚡ 异步基础: 利用异步编程知识理解现代Web框架")

## 1. HTTP协议基础

### 1.1 HTTP请求和响应结构

In [ ]:
print(f"\n📡 1. HTTP协议基础:")

@dataclass
class HTTPRequest:
    """HTTP请求数据结构"""
    method: str
    path: str
    headers: Dict[str, str]
    body: str = ""
    version: str = "HTTP/1.1"
    
    def to_string(self) -> str:
        """转换为HTTP请求字符串"""
        lines = [
            f"{self.method} {self.path} {self.version}",
        ]
        
        # 添加头部
        for key, value in self.headers.items():
            lines.append(f"{key}: {value}")
        
        lines.append("")  # 空行分隔头部和正文
        
        if self.body:
            lines.append(self.body)
        
        return "\r\n".join(lines)

@dataclass
class HTTPResponse:
    """HTTP响应数据结构"""
    status_code: int
    status_text: str
    headers: Dict[str, str]
    body: str = ""
    version: str = "HTTP/1.1"
    
    def to_string(self) -> str:
        """转换为HTTP响应字符串"""
        lines = [
            f"{self.version} {self.status_code} {self.status_text}",
        ]
        
        # 添加头部
        for key, value in self.headers.items():
            lines.append(f"{key}: {value}")
        
        lines.append("")  # 空行分隔头部和正文
        
        if self.body:
            lines.append(self.body)
        
        return "\r\n".join(lines)

print(f"   ✅ HTTP数据结构定义完成")
print(f"      - HTTPRequest: 封装HTTP请求信息")
print(f"      - HTTPResponse: 封装HTTP响应信息")

### 1.2 HTTP方法详解

In [ ]:
print(f"\n🔧 2. HTTP方法详解:")

class HTTPMethods:
    """HTTP方法常量和说明"""
    
    GET = "GET"
    POST = "POST"
    PUT = "PUT"
    DELETE = "DELETE"
    PATCH = "PATCH"
    HEAD = "HEAD"
    OPTIONS = "OPTIONS"
    
    @staticmethod
    def get_description(method: str) -> str:
        """获取HTTP方法描述"""
        descriptions = {
            "GET": "获取资源，安全且幂等",
            "POST": "创建资源，非安全且非幂等",
            "PUT": "完整更新资源，非安全但幂等",
            "DELETE": "删除资源，非安全但幂等",
            "PATCH": "部分更新资源，非安全且非幂等",
            "HEAD": "获取响应头，安全且幂等",
            "OPTIONS": "获取支持的方法，安全且幂等"
        }
        return descriptions.get(method, "未知方法")

# 演示HTTP方法
print(f"   📋 常用HTTP方法:")
for method in [HTTPMethods.GET, HTTPMethods.POST, HTTPMethods.PUT, HTTPMethods.DELETE]:
    desc = HTTPMethods.get_description(method)
    print(f"      - {method}: {desc}")

# 创建示例请求
get_request = HTTPRequest(
    method=HTTPMethods.GET,
    path="/api/users",
    headers={
        "Host": "localhost:8000",
        "User-Agent": "Python-Client/1.0",
        "Accept": "application/json"
    }
)

print(f"\n   📄 示例GET请求:")
print(f"      {get_request.to_string()}")

### 1.3 HTTP状态码

In [ ]:
print(f"\n📊 3. HTTP状态码:")

class HTTPStatus:
    """HTTP状态码常量"""
    
    # 2xx 成功
    OK = 200
    CREATED = 201
    ACCEPTED = 202
    NO_CONTENT = 204
    
    # 3xx 重定向
    MOVED_PERMANENTLY = 301
    FOUND = 302
    NOT_MODIFIED = 304
    
    # 4xx 客户端错误
    BAD_REQUEST = 400
    UNAUTHORIZED = 401
    FORBIDDEN = 403
    NOT_FOUND = 404
    METHOD_NOT_ALLOWED = 405
    
    # 5xx 服务器错误
    INTERNAL_SERVER_ERROR = 500
    NOT_IMPLEMENTED = 501
    BAD_GATEWAY = 502
    SERVICE_UNAVAILABLE = 503
    
    @staticmethod
    def get_status_text(status_code: int) -> str:
        """获取状态码文本"""
        status_texts = {
            200: "OK",
            201: "Created",
            202: "Accepted",
            204: "No Content",
            301: "Moved Permanently",
            302: "Found",
            304: "Not Modified",
            400: "Bad Request",
            401: "Unauthorized",
            403: "Forbidden",
            404: "Not Found",
            405: "Method Not Allowed",
            500: "Internal Server Error",
            501: "Not Implemented",
            502: "Bad Gateway",
            503: "Service Unavailable"
        }
        return status_texts.get(status_code, "Unknown")

# 创建示例响应
success_response = HTTPResponse(
    status_code=HTTPStatus.OK,
    status_text=HTTPStatus.get_status_text(HTTPStatus.OK),
    headers={
        "Content-Type": "application/json",
        "Content-Length": "27",
        "Server": "Python-Server/1.0"
    },
    body='{"message": "Hello, World!"}'
)

error_response = HTTPResponse(
    status_code=HTTPStatus.NOT_FOUND,
    status_text=HTTPStatus.get_status_text(HTTPStatus.NOT_FOUND),
    headers={
        "Content-Type": "application/json",
        "Content-Length": "35",
        "Server": "Python-Server/1.0"
    },
    body='{"error": "Resource not found"}'
)

print(f"   ✅ 成功响应示例:")
print(f"      {success_response.to_string()}")

print(f"   ❌ 错误响应示例:")
print(f"      {error_response.to_string()}")

## 2. Web框架基础

### 2.1 Web框架的作用和设计模式

In [ ]:
print(f"\n🏗️ 4. Web框架基础:")

class WebFramework:
    """简易Web框架基类"""
    
    def __init__(self):
        self.routes: Dict[Tuple[str, str], callable] = {}
        self.middleware: List[callable] = []
    
    def route(self, path: str, method: str = "GET"):
        """路由装饰器"""
        def decorator(func):
            self.routes[(method, path)] = func
            return func
        return decorator
    
    def add_middleware(self, middleware_func):
        """添加中间件"""
        self.middleware.append(middleware_func)
    
    def handle_request(self, request: HTTPRequest) -> HTTPResponse:
        """处理请求"""
        try:
            # 执行中间件
            for middleware in self.middleware:
                result = middleware(request)
                if isinstance(result, HTTPResponse):
                    return result
            
            # 查找路由
            route_key = (request.method, request.path)
            if route_key in self.routes:
                handler = self.routes[route_key]
                return handler(request)
            else:
                return HTTPResponse(
                    status_code=HTTPStatus.NOT_FOUND,
                    status_text=HTTPStatus.get_status_text(HTTPStatus.NOT_FOUND),
                    headers={"Content-Type": "text/plain"},
                    body="404 Not Found"
                )
        
        except Exception as e:
            return HTTPResponse(
                status_code=HTTPStatus.INTERNAL_SERVER_ERROR,
                status_text=HTTPStatus.get_status_text(HTTPStatus.INTERNAL_SERVER_ERROR),
                headers={"Content-Type": "text/plain"},
                body=f"500 Internal Server Error: {str(e)}"
            )

# 创建框架实例
app = WebFramework()

print(f"   ✅ Web框架基类创建完成")
print(f"      - 路由系统: 支持方法+路径路由")
print(f"      - 中间件支持: 可扩展的请求处理链")
print(f"      - 错误处理: 统一的异常处理机制")

### 2.2 路由系统实现

In [ ]:
print(f"\n🛣️ 5. 路由系统实现:")

# 定义路由处理函数
@app.route("/", "GET")
def home(request: HTTPRequest) -> HTTPResponse:
    """首页处理函数"""
    return HTTPResponse(
        status_code=HTTPStatus.OK,
        status_text=HTTPStatus.get_status_text(HTTPStatus.OK),
        headers={"Content-Type": "application/json"},
        body=json.dumps({"message": "Welcome to Python Web Framework!"})
    )

@app.route("/api/users", "GET")
def get_users(request: HTTPRequest) -> HTTPResponse:
    """获取用户列表"""
    users = [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ]
    
    return HTTPResponse(
        status_code=HTTPStatus.OK,
        status_text=HTTPStatus.get_status_text(HTTPStatus.OK),
        headers={"Content-Type": "application/json"},
        body=json.dumps(users)
    )

@app.route("/api/users", "POST")
def create_user(request: HTTPRequest) -> HTTPResponse:
    """创建新用户"""
    try:
        if request.headers.get("Content-Type") == "application/json":
            user_data = json.loads(request.body)
            # 模拟创建用户
            user_data["id"] = 999
            
            return HTTPResponse(
                status_code=HTTPStatus.CREATED,
                status_text=HTTPStatus.get_status_text(HTTPStatus.CREATED),
                headers={"Content-Type": "application/json"},
                body=json.dumps(user_data)
            )
        else:
            return HTTPResponse(
                status_code=HTTPStatus.BAD_REQUEST,
                status_text=HTTPStatus.get_status_text(HTTPStatus.BAD_REQUEST),
                headers={"Content-Type": "application/json"},
                body=json.dumps({"error": "Content-Type must be application/json"})
            )
    except json.JSONDecodeError:
        return HTTPResponse(
            status_code=HTTPStatus.BAD_REQUEST,
            status_text=HTTPStatus.get_status_text(HTTPStatus.BAD_REQUEST),
            headers={"Content-Type": "application/json"},
            body=json.dumps({"error": "Invalid JSON"})
        )

# 测试路由
print(f"   🧪 测试路由系统:")

# 测试首页
home_request = HTTPRequest(
    method="GET",
    path="/",
    headers={"Host": "localhost:8000"}
)
home_response = app.handle_request(home_request)
print(f"      首页响应: {home_response.status_code} - {home_response.body}")

# 测试获取用户
users_request = HTTPRequest(
    method="GET",
    path="/api/users",
    headers={"Host": "localhost:8000"}
)
users_response = app.handle_request(users_request)
print(f"      用户列表: {users_response.status_code} - 返回{len(json.loads(users_response.body))}个用户")

# 测试创建用户
create_request = HTTPRequest(
    method="POST",
    path="/api/users",
    headers={
        "Host": "localhost:8000",
        "Content-Type": "application/json"
    },
    body=json.dumps({"name": "David", "email": "david@example.com"})
)
create_response = app.handle_request(create_request)
print(f"      创建用户: {create_response.status_code} - {create_response.body}")

### 2.3 中间件系统

In [ ]:
print(f"\n🔗 6. 中间件系统:")

# 日志中间件
def logging_middleware(request: HTTPRequest):
    """请求日志中间件"""
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    print(f"      [{timestamp}] {request.method} {request.path}")
    # 继续处理请求
    return None

# 认证中间件
def auth_middleware(request: HTTPRequest):
    """简单的认证中间件"""
    # 检查Authorization头部
    auth_header = request.headers.get("Authorization")
    if not auth_header or not auth_header.startswith("Bearer "):
        return HTTPResponse(
            status_code=HTTPStatus.UNAUTHORIZED,
            status_text=HTTPStatus.get_status_text(HTTPStatus.UNAUTHORIZED),
            headers={"Content-Type": "application/json"},
            body=json.dumps({"error": "Unauthorized - Missing or invalid token"})
        )
    return None

# CORS中间件
def cors_middleware(request: HTTPRequest):
    """CORS处理中间件"""
    # 如果是OPTIONS请求，返回CORS预检响应
    if request.method == "OPTIONS":
        return HTTPResponse(
            status_code=HTTPStatus.OK,
            status_text=HTTPStatus.get_status_text(HTTPStatus.OK),
            headers={
                "Access-Control-Allow-Origin": "*",
                "Access-Control-Allow-Methods": "GET, POST, PUT, DELETE, OPTIONS",
                "Access-Control-Allow-Headers": "Content-Type, Authorization"
            }
        )
    return None

# 添加中间件到应用
app.add_middleware(logging_middleware)
app.add_middleware(cors_middleware)

# 创建需要认证的路由
@app.route("/api/protected", "GET")
def protected_data(request: HTTPRequest) -> HTTPResponse:
    """需要认证的受保护数据"""
    return HTTPResponse(
        status_code=HTTPStatus.OK,
        status_text=HTTPStatus.get_status_text(HTTPStatus.OK),
        headers={"Content-Type": "application/json"},
        body=json.dumps({"message": "This is protected data", "user": "authenticated"})
    )

# 创建受保护的应用实例
protected_app = WebFramework()
protected_app.add_middleware(logging_middleware)
protected_app.add_middleware(cors_middleware)
protected_app.add_middleware(auth_middleware)

@protected_app.route("/api/protected", "GET")
def protected_data_with_auth(request: HTTPRequest) -> HTTPResponse:
    """需要认证的受保护数据"""
    return HTTPResponse(
        status_code=HTTPStatus.OK,
        status_text=HTTPStatus.get_status_text(HTTPStatus.OK),
        headers={"Content-Type": "application/json"},
        body=json.dumps({"message": "This is protected data", "user": "authenticated"})
    )

print(f"   🧪 测试中间件系统:")

# 测试无认证的受保护路由
unauth_request = HTTPRequest(
    method="GET",
    path="/api/protected",
    headers={"Host": "localhost:8000"}
)
unauth_response = protected_app.handle_request(unauth_request)
print(f"      无认证请求: {unauth_response.status_code}")

# 测试有认证的受保护路由
auth_request = HTTPRequest(
    method="GET",
    path="/api/protected",
    headers={
        "Host": "localhost:8000",
        "Authorization": "Bearer valid-token-123"
    }
)
auth_response = protected_app.handle_request(auth_request)
print(f"      有认证请求: {auth_response.status_code}")

## 3. 同步 vs 异步Web框架

### 3.1 同步Web框架的特点

In [ ]:
print(f"\n⚡ 7. 同步 vs 异步Web框架:")

import threading
import time

class SyncWebFramework(WebFramework):
    """同步Web框架"""
    
    def __init__(self):
        super().__init__()
        self.thread_pool = []
    
    def handle_request_sync(self, request: HTTPRequest) -> HTTPResponse:
        """同步处理请求（阻塞）"""
        print(f"      [同步] 开始处理: {request.method} {request.path}")
        
        # 模拟阻塞操作（如数据库查询）
        time.sleep(0.1)  # 模拟I/O等待
        
        response = self.handle_request(request)
        
        print(f"      [同步] 完成处理: {request.method} {request.path}")
        return response
    
    def handle_concurrent_requests(self, requests: List[HTTPRequest]) -> List[HTTPResponse]:
        """使用线程池处理并发请求"""
        print(f"      [同步] 使用线程池处理{len(requests)}个请求")
        
        def process_request(req):
            return self.handle_request_sync(req)
        
        threads = []
        results = [None] * len(requests)
        
        for i, request in enumerate(requests):
            thread = threading.Thread(target=lambda i=i, req=request: results.__setitem__(i, process_request(req)))
            threads.append(thread)
            thread.start()
        
        for thread in threads:
            thread.join()
        
        return results

class AsyncWebFramework(WebFramework):
    """异步Web框架"""
    
    async def handle_request_async(self, request: HTTPRequest) -> HTTPResponse:
        """异步处理请求（非阻塞）"""
        print(f"      [异步] 开始处理: {request.method} {request.path}")
        
        # 模拟异步I/O操作
        await asyncio.sleep(0.1)  # 模拟异步I/O等待
        
        response = self.handle_request(request)
        
        print(f"      [异步] 完成处理: {request.method} {request.path}")
        return response
    
    async def handle_concurrent_requests(self, requests: List[HTTPRequest]) -> List[HTTPResponse]:
        """使用事件循环处理并发请求"""
        print(f"      [异步] 使用事件循环处理{len(requests)}个请求")
        
        tasks = [self.handle_request_async(request) for request in requests]
        results = await asyncio.gather(*tasks)
        
        return results

print(f"   ✅ 同步和异步框架创建完成")
print(f"      - SyncWebFramework: 基于线程池的并发处理")
print(f"      - AsyncWebFramework: 基于事件循环的并发处理")

### 3.2 性能对比测试

In [ ]:
print(f"\n🏁 8. 性能对比测试:")

# 创建测试请求
test_requests = [
    HTTPRequest(
        method="GET",
        path=f"/api/test/{i}",
        headers={"Host": "localhost:8000"}
    ) for i in range(5)
]

# 为测试添加路由
sync_app = SyncWebFramework()
async_app = AsyncWebFramework()

@sync_app.route("/api/test/{id}", "GET")
@async_app.route("/api/test/{id}", "GET")
def test_endpoint(request: HTTPRequest) -> HTTPResponse:
    """测试端点"""
    return HTTPResponse(
        status_code=HTTPStatus.OK,
        status_text=HTTPStatus.get_status_text(HTTPStatus.OK),
        headers={"Content-Type": "application/json"},
        body=json.dumps({"path": request.path, "timestamp": time.time()})
    )

# 性能测试
print(f"   📊 同步框架性能测试:")
start_time = time.time()
sync_results = sync_app.handle_concurrent_requests(test_requests)
sync_duration = time.time() - start_time
print(f"      处理时间: {sync_duration:.3f}秒")
print(f"      成功响应: {sum(1 for r in sync_results if r.status_code == 200)}")

print(f"\n   📊 异步框架性能测试:")
start_time = time.time()
async_results = await async_app.handle_concurrent_requests(test_requests)
async_duration = time.time() - start_time
print(f"      处理时间: {async_duration:.3f}秒")
print(f"      成功响应: {sum(1 for r in async_results if r.status_code == 200)}")

print(f"\n   📈 性能对比结果:")
if async_duration < sync_duration:
    improvement = ((sync_duration - async_duration) / sync_duration) * 100
    print(f"      异步框架比同步框架快 {improvement:.1f}%")
else:
    print(f"      在此测试场景下，同步框架表现更好")

print(f"\n   💡 异步框架优势:")
print(f"      - 更少的内存占用（无需创建线程）")
print(f"      - 更好的I/O密集型任务处理能力")
print(f"      - 更简单的并发编程模型")
print(f"      - 更适合高并发场景")

## 4. 现代Web框架介绍

### 4.1 主流Python Web框架对比

In [ ]:
print(f"\n🌟 9. 现代Web框架介绍:")

class WebFrameworkComparison:
    """Web框架对比分析"""
    
    frameworks = {
        "Django": {
            "type": "同步",
            "philosophy": "大而全， batteries-included",
            "use_case": "大型应用，内容管理系统",
            "async_support": "部分支持（Django 3.1+）",
            "learning_curve": "陡峭",
            "performance": "中等"
        },
        "Flask": {
            "type": "同步",
            "philosophy": "微框架，简洁灵活",
            "use_case": "小型应用，API服务",
            "async_support": "有限支持",
            "learning_curve": "平缓",
            "performance": "良好"
        },
        "FastAPI": {
            "type": "异步",
            "philosophy": "现代、快速、高性能",
            "use_case": "API服务，微服务",
            "async_support": "原生支持",
            "learning_curve": "中等",
            "performance": "极高"
        },
        "Starlette": {
            "type": "异步",
            "philosophy": "轻量级ASGI工具包",
            "use_case": "异步Web应用基础",
            "async_support": "原生支持",
            "learning_curve": "中等",
            "performance": "极高"
        },
        "Sanic": {
            "type": "异步",
            "philosophy": "类似Flask的异步框架",
            "use_case": "高性能API服务",
            "async_support": "原生支持",
            "learning_curve": "中等",
            "performance": "很高"
        }
    }
    
    @classmethod
    def print_comparison(cls):
        """打印框架对比"""
        print(f"   📊 Python Web框架对比:")
        print(f"      {'框架':<12} {'类型':<6} {'性能':<6} {'学习曲线':<8} {'适用场景'}")
        print(f"      {'-'*12} {'-'*6} {'-'*6} {'-'*8} {'-'*20}")
        
        for name, info in cls.frameworks.items():
            print(f"      {name:<12} {info['type']:<6} {info['performance']:<6} {info['learning_curve']:<8} {info['use_case']}")
    
    @classmethod
    def recommend_framework(cls, use_case: str) -> str:
        """根据使用场景推荐框架"""
        recommendations = {
            "大型企业应用": "Django",
            "内容管理系统": "Django",
            "RESTful API": "FastAPI",
            "微服务架构": "FastAPI",
            "高性能API": "FastAPI",
            "小型应用": "Flask",
            "快速原型": "Flask",
            "异步Web应用": "Starlette",
            "学习异步编程": "Starlette"
        }
        
        return recommendations.get(use_case, "请根据具体需求选择")

# 打印框架对比
WebFrameworkComparison.print_comparison()

print(f"\n   🎯 框架推荐:")
for use_case in ["RESTful API", "微服务架构", "大型企业应用", "快速原型"]:
    framework = WebFrameworkComparison.recommend_framework(use_case)
    print(f"      {use_case}: {framework}")

### 4.2 FastAPI简介和优势

In [ ]:
print(f"\n🚀 10. FastAPI简介和优势:")

class FastAPIFeatures:
    """FastAPI特性介绍"""
    
    features = {
        "高性能": {
            "description": "基于Starlette和Pydantic，性能与Node.js和Go相当",
            "benefit": "适合高并发API服务"
        },
        "快速编码": {
            "description": "减少约40%的代码量，减少bug",
            "benefit": "提高开发效率"
        },
        "减少错误": {
            "description": "基于类型提示，减少约40%的开发错误",
            "benefit": "提高代码质量"
        },
        "直观易用": {
            "description": "优秀的编辑器支持，自动补全和调试",
            "benefit": "降低学习成本"
        },
        "易于使用": {
            "description": "设计简洁，文档详细，学习曲线平缓",
            "benefit": "快速上手"
        },
        "标准兼容": {
            "description": "基于OpenAPI和JSON Schema，自动生成文档",
            "benefit": "标准化API开发"
        }
    }
    
    @classmethod
    def print_features(cls):
        """打印FastAPI特性"""
        print(f"   ⭐ FastAPI核心特性:")
        for feature, info in cls.features.items():
            print(f"      🔹 {feature}:")
            print(f"         描述: {info['description']}")
            print(f"         优势: {info['benefit']}")
            print()
    
    @classmethod
    def async_advantages(cls):
        """异步优势说明"""
        print(f"   ⚡ FastAPI异步优势:")
        print(f"      - 原生支持async/await语法")
        print(f"      - 自动处理并发请求，无需线程池")
        print(f"      - 适合I/O密集型任务（数据库、网络请求）")
        print(f"      - 更好的资源利用率")
        print(f"      - 与LangChain异步特性完美结合")

# 打印FastAPI特性
FastAPIFeatures.print_features()
FastAPIFeatures.async_advantages()

print(f"\n   🎯 为什么选择FastAPI:")
print(f"      1. 异步原生支持，性能卓越")
print(f"      2. 类型提示驱动，开发体验优秀")
print(f"      3. 自动API文档生成")
print(f"      4. 现代Python特性支持")
print(f"      5. 与LangChain生态完美集成")

## 5. LangChain Web应用场景

### 5.1 Web框架在LangChain中的作用

In [ ]:
print(f"\n🤖 11. LangChain Web应用场景:")

class LangChainWebScenarios:
    """LangChain Web应用场景分析"""
    
    scenarios = {
        "对话API服务": {
            "description": "提供HTTP接口的AI对话服务",
            "tech_stack": "FastAPI + 异步处理",
            "key_features": ["实时对话", "流式响应", "会话管理"],
            "async_benefits": ["并发对话处理", "非阻塞响应", "资源优化"]
        },
        "RAG问答系统": {
            "description": "基于检索增强生成的问答服务",
            "tech_stack": "FastAPI + 向量数据库",
            "key_features": ["文档检索", "智能问答", "知识库管理"],
            "async_benefits": ["并行检索", "异步生成", "高并发查询"]
        },
        "AI工作流引擎": {
            "description": "可视化的AI工作流编排平台",
            "tech_stack": "FastAPI + WebSocket + 任务队列",
            "key_features": ["工作流设计", "实时监控", "任务调度"],
            "async_benefits": ["实时通信", "异步任务", "并发执行"]
        },
        "智能客服系统": {
            "description": "企业级智能客服解决方案",
            "tech_stack": "FastAPI + WebSocket + 消息队列",
            "key_features": ["多渠道接入", "智能路由", "人工协作"],
            "async_benefits": ["实时通信", "负载均衡", "故障恢复"]
        }
    }
    
    @classmethod
    def print_scenarios(cls):
        """打印应用场景"""
        print(f"   🎯 LangChain Web应用场景:")
        for name, scenario in cls.scenarios.items():
            print(f"\n      📱 {name}:")
            print(f"         描述: {scenario['description']}")
            print(f"         技术栈: {scenario['tech_stack']}")
            print(f"         核心功能: {', '.join(scenario['key_features'])}")
            print(f"         异步优势: {', '.join(scenario['async_benefits'])}")
    
    @classmethod
    def architecture_benefits(cls):
        """架构优势说明"""
        print(f"\n   🏗️ Web架构对LangChain的价值:")
        print(f"      1. API标准化: 通过HTTP接口暴露LangChain能力")
        print(f"      2. 服务化部署: 将AI功能作为微服务提供")
        print(f"      3. 前后端分离: 支持多种客户端接入")
        print(f"      4. 扩展性设计: 便于功能扩展和性能优化")
        print(f"      5. 生产就绪: 完整的监控、日志、部署方案")

# 打印应用场景
LangChainWebScenarios.print_scenarios()
LangChainWebScenarios.architecture_benefits()

### 5.2 异步Web框架与LangChain的结合

In [ ]:
print(f"\n🔗 12. 异步Web框架与LangChain的结合:")

class AsyncLangChainIntegration:
    """异步Web框架与LangChain集成示例"""
    
    @staticmethod
    async def simulate_langchain_chat(message: str) -> str:
        """模拟LangChain异步对话"""
        # 模拟AI处理时间
        await asyncio.sleep(0.1)
        
        # 模拟响应生成
        responses = [
            f"这是对'{message}'的AI回复",
            f"我理解您说的'{message}'，让我为您解答...",
            f"关于'{message}'，我的看法是..."
        ]
        
        import random
        return random.choice(responses)
    
    @staticmethod
    async def simulate_langchain_rag(query: str) -> Dict[str, Any]:
        """模拟LangChain RAG查询"""
        # 模拟文档检索
        await asyncio.sleep(0.05)
        
        # 模拟检索到的文档
        documents = [
            f"关于{query}的相关文档1...",
            f"关于{query}的相关文档2...",
            f"关于{query}的相关文档3..."
        ]
        
        # 模拟答案生成
        await asyncio.sleep(0.1)
        answer = f"基于检索到的文档，关于'{query}'的答案是..."
        
        return {
            "query": query,
            "documents": documents,
            "answer": answer,
            "processing_time": 0.15
        }
    
    @classmethod
    async def demonstrate_concurrent_processing(cls):
        """演示并发处理能力"""
        print(f"      🚀 演示异步并发处理:")
        
        # 模拟多个并发请求
        messages = ["你好", "天气怎么样", "推荐一本书", "解释机器学习", "如何学习Python"]
        
        start_time = time.time()
        
        # 并发处理聊天请求
        chat_tasks = [cls.simulate_langchain_chat(msg) for msg in messages]
        chat_results = await asyncio.gather(*chat_tasks)
        
        chat_duration = time.time() - start_time
        
        print(f"         并发处理{len(messages)}个聊天请求: {chat_duration:.3f}秒")
        
        # 并发处理RAG查询
        queries = ["Python基础", "机器学习算法", "Web开发", "数据库设计", "API设计"]
        
        start_time = time.time()
        
        rag_tasks = [cls.simulate_langchain_rag(query) for query in queries]
        rag_results = await asyncio.gather(*rag_tasks)
        
        rag_duration = time.time() - start_time
        
        print(f"         并发处理{len(queries)}个RAG查询: {rag_duration:.3f}秒")
        
        # 显示结果
        print(f"\n         📊 处理结果:")
        print(f"            聊天响应: {len(chat_results)}个")
        print(f"            RAG查询: {len(rag_results)}个")
        print(f"            平均响应时间: {max(chat_duration, rag_duration)/len(messages):.3f}秒/请求")

# 演示集成效果
print(f"   🤖 异步Web框架与LangChain集成演示:")

# 单个请求测试
print(f"\n   📝 单个请求测试:")
start_time = time.time()
chat_response = await AsyncLangChainIntegration.simulate_langchain_chat("你好，请介绍一下异步编程")
chat_time = time.time() - start_time
print(f"      聊天响应: {chat_time:.3f}秒 - {chat_response}")

start_time = time.time()
rag_response = await AsyncLangChainIntegration.simulate_langchain_rag("Python异步编程")
rag_time = time.time() - start_time
print(f"      RAG查询: {rag_time:.3f}秒 - 找到{len(rag_response['documents'])}个文档")

# 并发处理演示
await AsyncLangChainIntegration.demonstrate_concurrent_processing()

## 学习总结

### 📚 知识点回顾

In [ ]:
print(f"\n📚 学习总结:")
print("=" * 50)

class LearningSummary:
    """学习内容总结"""
    
    topics_covered = [
        "HTTP协议基础（请求/响应结构、方法、状态码）",
        "Web框架设计原理（路由系统、中间件、错误处理）",
        "同步vs异步Web框架（性能对比、适用场景）",
        "主流Python Web框架对比（Django、Flask、FastAPI等）",
        "FastAPI特性和优势（高性能、类型提示、自动文档）",
        "LangChain Web应用场景（对话API、RAG系统、工作流引擎）",
        "异步Web框架与LangChain的完美结合"
    ]
    
    practical_skills = [
        "能够设计和实现简单的HTTP请求/响应处理",
        "理解Web框架的核心组件和工作原理",
        "掌握同步和异步编程在Web开发中的差异",
        "能够根据项目需求选择合适的Web框架",
        "理解FastAPI的设计理念和应用场景",
        "能够设计基于LangChain的Web应用架构",
        "掌握异步Web开发的核心概念和最佳实践"
    ]
    
    @classmethod
    def print_summary(cls):
        """打印学习总结"""
        print(f"✅ 已掌握知识点:")
        for i, topic in enumerate(cls.topics_covered, 1):
            print(f"   {i}. {topic}")
        
        print(f"\n🎯 实践技能获得:")
        for i, skill in enumerate(cls.practical_skills, 1):
            print(f"   {i}. {skill}")
    
    @classmethod
    def next_steps(cls):
        """下一步学习计划"""
        print(f"\n🚀 下一步学习计划:")
        print(f"   1. 深入学习FastAPI框架的实际应用")
        print(f"   2. 掌握RESTful API设计原则和最佳实践")
        print(f"   3. 学习WebSocket实时通信技术")
        print(f"   4. 了解异步数据库操作和ORM")
        print(f"   5. 掌握Web安全和身份认证机制")
        print(f"   6. 学习前后端分离和API文档生成")
        print(f"   7. 了解微服务架构设计模式")
        print(f"   8. 掌握Web应用部署和运维知识")

# 打印学习总结
LearningSummary.print_summary()
LearningSummary.next_steps()

print(f"\n🎉 HTTP协议基础与Web框架入门 学习完成!")
print(f"🔗 已为后续的FastAPI实战和LangChain Web开发奠定坚实基础")
print(f"⚡ 异步编程基础将帮助您更好地理解现代Web框架的设计理念")